<a href="https://colab.research.google.com/github/hayannn/AIFFEL_STUDY/blob/main/8_4_%E1%84%90%E1%85%A6%E1%86%A8%E1%84%89%E1%85%B3%E1%84%90%E1%85%B3_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8___20_%E1%84%82%E1%85%B2%E1%84%89%E1%85%B3%E1%84%80%E1%85%B3%E1%84%85%E1%85%AE%E1%86%B8_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro
- 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘 : 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등
- 카운트 기반과 TF-IDF 기반 벡터화를 차례로 적용해 예측 성능 비교
- 피쳐 벡터화를 위한 파라미터와 GridSearchCV 기반 하이퍼 파라미터 튜닝
- 사이킷런의 Pipeline 객체를 통한 피쳐 벡터화 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝을 한꺼번에 수행하는 방법 소개

## 텍스트 정규화
- fetch_20newsgroups() : 인터넷에서 로컬 컴퓨터로 데이터 내려받기 -> 메모리로 데이터를 로딩
- 파이썬 딕셔너리와 유사한 Bunch 객체 반환

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

- key 값 확인

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


- Target 클래스 구성 확인

In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


- 개별 데이터가 텍스트로 어떻게 구성되어 있는지 확인
  - 데이터를 1개만 추출해 값 확인

In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 뉴스그룹 기사만이 아닌 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보를 가지고 있음
  - 제목 제외 다른 정보는 제거 -> 다른 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문
  - 이 피쳐들을 포함하면 -> ML 알고리즘을 적용해도 상당히 높은 예측 성능을 나타내지만, 지금 학습하려는 텍스트 분석의 의도를 벗어나기 때문에 **순수 텍스트만으로 이뤄진 기사 내용으로 어떤 뉴스그룹에 속하는지** 분류!

<br>

- `remove` 파라미터를 이용해 뉴스그룹 기사 헤더, 푸터 등 제거 가능
- feath_20newsgroups()는 `subset` 파라미터를 이용해 학습 데이터셋과 테스트 데이터셋을 분리해서 내려받을 수 있음

In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용(Train) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

# subset='test'으로 테스트(Test) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))

<class 'list'>
학습 데이터 크기 11314 , 테스트 데이터 크기 7532


## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- CountVectorizer를 이용해 학습 데이터 텍스트를 피쳐 벡터화
  - 테스트 데이터 주의 사항
    - 테스트 데이터에서 CountVectorizer 적용시 반드시 **학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체**로 테스트 데이터를 변환(transform)해야 함!
    - 그래야 학습 시 설정된 CountVectorizer의 피쳐 개수와 테스트 데이터를 CountVectorizer로 변환할 피쳐 개수가 같아짐
    - 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform()을 이용해 변환
    
    <br>

    - **테스트 데이터 피쳐 벡터화 시 fit_transform() 사용하면 안된다는 점도 유의**
    - 이유는 동일


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환
cnt_vect = CountVectorizer()

cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


- 단어가 101631개로 만들어짐!

<br>

- 로지스틱 회귀 적용해 뉴스그룹에 대한 분류를 예측
  - count 기반

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# LogisticRegression을 이용하여 학습/예측/평가
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.617


- TF-IDF 기반
  - 단순 카운드 기반보다 훨씬 높은 예측 정확도 제공

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.678


- 텍스트 분석에서 머신러닝 모델 성능을 향상시키는 중요한 2가지 방법
  - 최적 ML 알고리즘 선택
  - 최상 피쳐 전처리 수행

In [9]:
# stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.690


- GridSearchCV로 로지스틱 회귀의 하이퍼 파라미터 최적화 수행
  - C가 10일 때 최적
  - 테스트 데이터셋에 적용해 약 0.704 수치(이전보다 약간 향상)

In [10]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 (CV는 3 Fold셋으로 설정)
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C값으로 학습된 grid_cv로 예측 수행 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.704


## 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합
- ML에서 Pipeline이란
  - 데이터 가공, 변환 등의 전처리와 알고리즘 적용을 한꺼번에 스트림 기반으로 처리한다는 의미
  - 데이터 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드 생성 가능
  - 대용량 데이터 피쳐 벡터화 결과를 스트림 기반에서 바로 스트림 기반에서 바로 ML 알고리즘 데이터로 입력할 수 있어 수행 시간 절약 가능

- 사이킷런 파이프라인
  - 텍스트 기반 피쳐 벡터화, 모든 데이터 전처리 작업, Estimator 결합 가능
  - ex) 스케일링 또는 벡터 정규화, PCA 등의 변환 작업 + 분류, 회귀 등의 Estimator를 한번에 결합
  - `pipeline = Pipeline([...` : TfidfVectorizer 객체를 tfidf_vect라는 객체 변수명으로, LogisticRegression 객체를 lr_clf라는 객체 변수명으로 생성 -> 이 2개의 객체를 파이프라인으로 연결하는 Pipeline 객체 pipline을 생성한다는 의미

In [11]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect 객체명으로, LogisticRegression객체를 lr_clf 객체명으로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

# 별도의 TfidfVectorizer객체의 fit_transform()과 LogisticRegression의 fit(), predict()가 필요 없음
# pipeline의 fit() 과 predict()만으로 한꺼번에 Feature Vectorization과 ML 학습/예측 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.704


- GridSearchCV에 Pipeline을 입력하며 TfidfVectorizer의 파라미터와 Logistic Regression의 하이퍼 파라미터를 함께 최적화
- Pipeline + GridSearchCV 적용 시 유의점
  - 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모된다는 점

In [12]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression(solver='liblinear'))
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼 파라미터 이름과 값을 설정
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1, 5, 10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.7550828826229531
Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.702


- TfidVectorizer 객체의 max_df가 700, ngram_range 파라미터가 (1,2)로 피쳐 벡터화된 데이터셋에
- LogisticRegression의 C에 10을 적용해 예측 분류를 수행할 때 가장 좋음!
- 정확도의 경우에는 크게 향상되지 않음